In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:90% !important; }</style>"))

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
sys.path.insert(0, '../')

import numpy as np
import tensorflow as tf
import os, sys, time, pickle, functools
import utils, optimization, metrics, plot, data, evaluation

import plot
from plot import get_animation

from model import WGanModel, LapGanModel
from gan import CosmoGAN, UpscaleCosmoGAN
import blocks
from blocks import *

import sys
sys.path.insert(0, '../')

import numpy as np
import tensorflow as tf
import os
import data, utils
from model import upscale_WGAN_pixel_CNN, WGanModel
from gan import CosmoGAN, GAN

/home/ankit/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
try_resume = True
Mpch = 350

# Generate samples - more_data/upscaling_GAN_3d_gen_8_disc_6_32_new_trans

In [4]:
ns = 32
time_str = 'upscaling_GAN_3d_gen_8_disc_6_' + str(ns) + '_new_trans'

global_path = '../../up_scale_GAN_results/upscaling_GAN_3d/inception/upscaling_GAN_3d_' + str(ns) + '_gen_8_disc_6_new_trans/'
name = 'inception_upscaling_GAN_3d_{}'.format(ns)

params = dict()
params['save_dir'] = global_path + name + '_' + time_str + 'checkpoints/'

resume, params = utils.test_resume(try_resume, params)

params['cosmology']['backward_map'] = data.fmap.backward
params['cosmology']['forward_map'] = data.fmap.forward

for ind, item in enumerate(params.items()):
    print(ind, " : ", item[0], " - ", item[1])
print(resume)

params['summary_dir'] = global_path + params['name'] + '_' + time_str +'summary/'
params['save_dir'] = global_path + params['name'] + '_' + time_str + 'checkpoints/'

print(params['summary_dir'])
print(params['save_dir'])

Resume, the training will start from the last iteration!
0  :  generator  -  {'stride': [1, 1, 1, 1, 1, 1, 1, 1], 'is_3d': True, 'inception': True, 'full': [], 'y_layer': 0, 'latent_dim': 32768, 'non_lin': None, 'batch_norm': [False, False, False, False, False, False, False], 'summary': True, 'nfilter': [32, 32, 64, 64, 64, 32, 32, 1], 'one_pixel_mapping': []}
1  :  optimization  -  {'disc_learning_rate': 3e-05, 'enc_optimizer': 'rmsprop', 'beta1': 0.9, 'disc_optimizer': 'rmsprop', 'batch_size': 8, 'beta2': 0.999, 'enc_learning_rate': 3e-05, 'gen_optimizer': 'rmsprop', 'gen_learning_rate': 3e-05, 'n_critic': 10, 'epsilon': 1e-08, 'gamma_gp': 5, 'epoch': 2000}
2  :  curr_counter  -  43000
3  :  resume  -  False
4  :  summary_dir  -  ../saved_result/inception_upscaling_GAN_3d_32_upscaling_GAN_3d_gen_8_disc_6_32_new_transsummary/
5  :  sum_every  -  200
6  :  discriminator  -  {'stride': [2, 2, 2, 1, 1, 1], 'is_3d': True, 'inception': True, 'full': [64, 16], 'non_lin': None, 'batch_norm':

# Load Real Data

In [11]:
dataset = data.load.load_dataset_file(spix=256, resolution=256,Mpch=Mpch, forward_map=params['cosmology']['forward_map'], patch=True, is_3d=True)
real_nbody = dataset.get_all_data()
real_nbody_raw = params['cosmology']['backward_map'](real_nbody)
print("Nbody dataset shape: ", np.shape(real_nbody))

Nbody dataset shape:  (2, 256, 256, 256, 8)


In [12]:
wgan = UpscaleCosmoGAN(params, upscale_WGAN_pixel_CNN, is_3d=True)

Generator 
--------------------------------------------------
     The input z is of size (?, 32768)
     The input y is of size (?, 32, 32, 32, 7)
     Reshape z to (?, 32, 32, 32, 1)
     Merge input y of size(?, 32, 32, 32, 7)
     Concat x and y to (?, 32, 32, 32, 8)
     0 Inception(1x1,3x3,5x5) layer with 32 channels
         Size of the variables: (?, 32, 32, 32, 96)
     1 Inception(1x1,3x3,5x5) layer with 32 channels
         Size of the variables: (?, 32, 32, 32, 96)
     2 Inception(1x1,3x3,5x5) layer with 64 channels
         Size of the variables: (?, 32, 32, 32, 192)
     3 Inception(1x1,3x3,5x5) layer with 64 channels
         Size of the variables: (?, 32, 32, 32, 192)
     4 Inception(1x1,3x3,5x5) layer with 64 channels
         Size of the variables: (?, 32, 32, 32, 192)
     5 Inception(1x1,3x3,5x5) layer with 32 channels
         Size of the variables: (?, 32, 32, 32, 96)
     6 Inception(1x1,3x3,5x5) layer with 32 channels
         Size of the variables: (?, 32, 32

In [13]:
num_samples = 2
gen_samples = wgan.upscale_image(num_samples=num_samples, resolution=256, checkpoint='38000')
gen_samples_raw = params['cosmology']['backward_map'](gen_samples)
print('Done')

 [*] Reading checkpoints...
INFO:tensorflow:Restoring parameters from ../../up_scale_GAN_results/upscaling_GAN_3d/inception/upscaling_GAN_3d_32_gen_8_disc_6_new_trans/inception_upscaling_GAN_3d_32_upscaling_GAN_3d_gen_8_disc_6_32_new_transcheckpoints/inception_upscaling_GAN_3d_32-38000
Total number of patches = 8*8*8 = 512
Current patch: column=1, row=1, height=1
gen_sample: min=-0.15792058408260345 max=2.9641852378845215


Current patch: column=2, row=1, height=1
gen_sample: min=-0.16070862114429474 max=3.047560214996338


Current patch: column=3, row=1, height=1
gen_sample: min=-0.17803363502025604 max=3.4466209411621094


Current patch: column=4, row=1, height=1
gen_sample: min=-0.27765652537345886 max=3.055788516998291


Current patch: column=5, row=1, height=1
gen_sample: min=-0.16891570389270782 max=2.729020595550537


Current patch: column=6, row=1, height=1
gen_sample: min=-0.1293402761220932 max=2.568725109100342


Current patch: column=7, row=1, height=1
gen_sample: min=-0.09

gen_sample: min=-0.11449550092220306 max=2.769817590713501


Current patch: column=7, row=2, height=2
gen_sample: min=-0.07201690971851349 max=2.7752110958099365


Current patch: column=8, row=2, height=2
gen_sample: min=-0.17090249061584473 max=3.1199681758880615


Current patch: column=1, row=3, height=2
gen_sample: min=-0.0901147872209549 max=2.993525505065918


Current patch: column=2, row=3, height=2
gen_sample: min=-0.09221826493740082 max=2.8906173706054688


Current patch: column=3, row=3, height=2
gen_sample: min=-0.10200634598731995 max=2.6690866947174072


Current patch: column=4, row=3, height=2
gen_sample: min=-0.07896830141544342 max=2.8970694541931152


Current patch: column=5, row=3, height=2
gen_sample: min=-0.09636305272579193 max=2.6849422454833984


Current patch: column=6, row=3, height=2
gen_sample: min=-0.09283305704593658 max=2.932990074157715


Current patch: column=7, row=3, height=2
gen_sample: min=-0.13622666895389557 max=3.533529281616211


Current patch: c

gen_sample: min=-0.08348943293094635 max=3.730611801147461


Current patch: column=8, row=4, height=3
gen_sample: min=-0.23587505519390106 max=2.760564088821411


Current patch: column=1, row=5, height=3
gen_sample: min=-0.20476357638835907 max=3.1823461055755615


Current patch: column=2, row=5, height=3
gen_sample: min=-0.1442645788192749 max=2.8851821422576904


Current patch: column=3, row=5, height=3
gen_sample: min=-0.17871560156345367 max=2.784343719482422


Current patch: column=4, row=5, height=3
gen_sample: min=-0.1043088287115097 max=2.7942652702331543


Current patch: column=5, row=5, height=3
gen_sample: min=-0.2923121750354767 max=2.8545212745666504


Current patch: column=6, row=5, height=3
gen_sample: min=-0.1520140916109085 max=2.670347213745117


Current patch: column=7, row=5, height=3
gen_sample: min=-0.17419494688510895 max=2.780486583709717


Current patch: column=8, row=5, height=3
gen_sample: min=-0.15084366500377655 max=2.7645554542541504


Current patch: colum

gen_sample: min=-0.17049770057201385 max=2.681619167327881


Current patch: column=1, row=7, height=4
gen_sample: min=-0.1398373693227768 max=3.820664882659912


Current patch: column=2, row=7, height=4
gen_sample: min=-0.11548073589801788 max=3.0449280738830566


Current patch: column=3, row=7, height=4
gen_sample: min=-0.15525861084461212 max=3.3230574131011963


Current patch: column=4, row=7, height=4
gen_sample: min=-0.10958471894264221 max=2.9742634296417236


Current patch: column=5, row=7, height=4
gen_sample: min=-0.09046338498592377 max=2.80983567237854


Current patch: column=6, row=7, height=4
gen_sample: min=-0.17023871839046478 max=2.999164581298828


Current patch: column=7, row=7, height=4
gen_sample: min=-0.11949814856052399 max=3.1604833602905273


Current patch: column=8, row=7, height=4
gen_sample: min=-0.19791974127292633 max=3.359433650970459


Current patch: column=1, row=8, height=4
gen_sample: min=-0.15121899545192719 max=3.315552234649658


Current patch: colu

gen_sample: min=-0.0993424504995346 max=2.9053802490234375


Current patch: column=2, row=1, height=6
gen_sample: min=-0.16875328123569489 max=2.9030981063842773


Current patch: column=3, row=1, height=6
gen_sample: min=-0.10597002506256104 max=2.6632351875305176


Current patch: column=4, row=1, height=6
gen_sample: min=-0.08389367163181305 max=2.6144278049468994


Current patch: column=5, row=1, height=6
gen_sample: min=-0.15213121473789215 max=2.6101365089416504


Current patch: column=6, row=1, height=6
gen_sample: min=-0.12490001320838928 max=2.3814892768859863


Current patch: column=7, row=1, height=6
gen_sample: min=-0.07309438288211823 max=2.8931307792663574


Current patch: column=8, row=1, height=6
gen_sample: min=-0.31698015332221985 max=3.4199929237365723


Current patch: column=1, row=2, height=6
gen_sample: min=-0.12174442410469055 max=3.1413302421569824


Current patch: column=2, row=2, height=6
gen_sample: min=-0.11953462660312653 max=2.880005121231079


Current patch

gen_sample: min=-0.15597917139530182 max=2.963465690612793


Current patch: column=3, row=3, height=7
gen_sample: min=-0.09743449091911316 max=2.6258888244628906


Current patch: column=4, row=3, height=7
gen_sample: min=-0.07543274760246277 max=2.6486363410949707


Current patch: column=5, row=3, height=7
gen_sample: min=-0.16083502769470215 max=2.3676979541778564


Current patch: column=6, row=3, height=7
gen_sample: min=-0.0815676897764206 max=2.587836980819702


Current patch: column=7, row=3, height=7
gen_sample: min=-0.06637789309024811 max=2.8296003341674805


Current patch: column=8, row=3, height=7
gen_sample: min=-0.09570762515068054 max=3.209117889404297


Current patch: column=1, row=4, height=7
gen_sample: min=-0.1385691910982132 max=2.9943699836730957


Current patch: column=2, row=4, height=7
gen_sample: min=-0.09675373136997223 max=2.2864222526550293


Current patch: column=3, row=4, height=7
gen_sample: min=-0.06593358516693115 max=3.332902193069458


Current patch: co

gen_sample: min=-0.13232864439487457 max=2.484873056411743


Current patch: column=4, row=5, height=8
gen_sample: min=-0.1111062616109848 max=3.0761566162109375


Current patch: column=5, row=5, height=8
gen_sample: min=-0.2372453659772873 max=2.8111135959625244


Current patch: column=6, row=5, height=8
gen_sample: min=-0.14344479143619537 max=2.882262706756592


Current patch: column=7, row=5, height=8
gen_sample: min=-0.17076337337493896 max=3.115324020385742


Current patch: column=8, row=5, height=8
gen_sample: min=-0.11676304042339325 max=3.499912738800049


Current patch: column=1, row=6, height=8
gen_sample: min=-0.12483350932598114 max=2.9576833248138428


Current patch: column=2, row=6, height=8
gen_sample: min=-0.09269018471240997 max=3.165834426879883


Current patch: column=3, row=6, height=8
gen_sample: min=-0.26976972818374634 max=2.51755952835083


Current patch: column=4, row=6, height=8
gen_sample: min=-0.09387919306755066 max=2.671755313873291


Current patch: column

In [14]:
utils.save_hdf5(gen_samples, 'Samples/inception/upscaling_GAN_3d_32_gen_8_disc_6_new_trans/2_256_samples_ckpt_38000.h5')

# Generate samples - 32->64->256

In [15]:
ln_fwd = data.fmap.log_norm_forward
ln_bkd = data.fmap.log_norm_backward

l_fwd = data.fmap.forward
l_bkd = data.fmap.backward

m_fwd = data.fmap.medical_forward
m_bkd = data.fmap.medical_backward

fwd = m_fwd
bkd = m_bkd

def non_lin(x):
		return (tf.nn.tanh(x) + 1.0)/2.0

## 0->32

In [18]:
ns = 32
#time_str = '32_32_32_gen_8_disc_6_new_trans'
#time_str = '32_32_32_log_norm'
time_str = '0_to_32'

#global_path = '../../up_scale_GAN_results/upscaling_GAN_3d/bigstat/WGAN_32_32_32_new_trans/'
#global_path = '../../up_scale_GAN_results/upscaling_GAN_3d/final/32_32_32_log_norm/'
global_path = '../../up_scale_GAN_results/upscaling_GAN_3d/medical/0_32/'

name = 'WGAN'

params_32 = dict()
params_32['save_dir'] = global_path + name + '_' + time_str + 'checkpoints/'

resume, params_32 = utils.test_resume(try_resume, params_32)
print(params_32['curr_counter'])

params_32['summary_dir'] = global_path + params_32['name'] + '_' + time_str +'summary/'
params_32['save_dir'] = global_path + params_32['name'] + '_' + time_str + 'checkpoints/'

print(params_32['summary_dir'])
print(params_32['save_dir'])

Resume, the training will start from the last iteration!
3600
../../up_scale_GAN_results/upscaling_GAN_3d/medical/0_32/WGAN_0_to_32summary/
../../up_scale_GAN_results/upscaling_GAN_3d/medical/0_32/WGAN_0_to_32checkpoints/


In [19]:
wgan_32 = CosmoGAN(params_32, WGanModel, is_3d=True)
num_sampled = 8
gen_samples_32, _ = wgan_32.generate(N=num_sampled, checkpoint=None) #27000 for nbody log linear transform
gen_samples_32 = np.squeeze(gen_samples_32)
#utils.save_hdf5(gen_samples_32, 'Samples/final/32_32_32_log_norm/8_32_samples_ckpt_27k.h5')
utils.save_hdf5(gen_samples_32, 'Samples/medical/0_32/8_32_samples_ckpt_3600.h5')

Generator 
--------------------------------------------------
     The input is of size (?, 100)
     0 Full layer with 64 outputs
         Size of the variables: (?, 64)
     Reshape to (?, 2, 2, 2, 8)
     0 Inception deconv(1x1,3x3,5x5) layer with 8 channels
         Size of the variables: (?, 4, 4, 4, 24)
     1 Inception deconv(1x1,3x3,5x5) layer with 32 channels
         Size of the variables: (?, 8, 8, 8, 96)
     2 Inception deconv(1x1,3x3,5x5) layer with 64 channels
         Size of the variables: (?, 16, 16, 16, 192)
     3 Inception deconv(1x1,3x3,5x5) layer with 64 channels
         Size of the variables: (?, 32, 32, 32, 192)
     4 Inception deconv(1x1,3x3,5x5) layer with 64 channels
         Size of the variables: (?, 32, 32, 32, 192)
     5 Inception deconv(1x1,3x3,5x5) layer with 32 channels
         Size of the variables: (?, 32, 32, 32, 96)
     6 Inception deconv(1x1,3x3,5x5) layer with 32 channels
         Size of the variables: (?, 32, 32, 32, 96)
     7 Inception 

 [*] Reading checkpoints...
INFO:tensorflow:Restoring parameters from ../../up_scale_GAN_results/upscaling_GAN_3d/medical/0_32/WGAN_0_to_32checkpoints/WGAN-3600
Checkpoint succesfully loaded!


## 32->64

## Fake 32

In [5]:
#gen_samples_32 = utils.load_hdf5('Samples/bigstat/WGAN_32_32_32_new_trans/8_32_samples_ckpt_163k.h5')[:2]
#gen_samples_32 = utils.load_hdf5('Samples/final/32_32_32_log_norm/8_32_samples_ckpt_27k.h5')[:2]
gen_samples_32 = utils.load_hdf5('Samples/medical/0_32/8_32_samples_ckpt_23k.h5')[:2]
print("gen_samples_32 shape=", gen_samples_32.shape)
gen_samples_32 = np.expand_dims(gen_samples_32, axis=4)
print('downsampled shape=', gen_samples_32.shape)

gen_samples_32 shape= (2, 32, 32, 32)
downsampled shape= (2, 32, 32, 32, 1)


## Real 64

In [6]:
#dataset_64 = data.load.load_dataset_file(spix=64, resolution=256, Mpch=350, scaling=4, augmentation=False, shuffle=False, forward_map=fwd, patch=False, is_3d=True)
dataset_64 = data.load.load_medical_dataset(spix=64, resolution=256, scaling=4, augmentation=False, shuffle=False, forward_map=fwd, patch=False, is_3d=True)
real_nbody_64 = dataset_64.get_all_data()
print("Nbody dataset shape: ", np.shape(real_nbody_64))

num_samples = 2
downsampled_32 = real_nbody_64[:num_samples, :, :, :]
downsampled_32 = blocks.downsample(downsampled_32, s=2, is_3d=True)
downsampled_32 = np.expand_dims(downsampled_32, axis=4)
print('downsampled_32 shape=', downsampled_32.shape)

Number of histograms=10
Tensor down_sampler_out_3d_4:0 not found, hence creating the Op.
Loaded 5 histograms for iterator _get_total_num_samples!
Loaded 5 histograms for iterator _get_total_num_samples!
Nbody dataset shape:  (10, 64, 64, 64)
Tensor down_sampler_out_3d_2:0 not found, hence creating the Op.
downsampled_32 shape= (2, 32, 32, 32, 1)


In [7]:
print(np.min(gen_samples_32), np.max(gen_samples_32))
print(np.min(downsampled_32), np.max(downsampled_32))

1.1416289 4.470638
0.9672947 3.7825904


In [8]:
ns = 32
#time_str = 'upscaling_GAN_3d_gen_8_disc_6_32_new_trans'
#time_str = 'log_norm'
time_str = '32_to_64'

#global_path = '../../up_scale_GAN_results/upscaling_GAN_3d/bigstat/inception_32_to_64_upsampled_upscaling_GAN_3d_32_gen_8_disc_6_new_trans/'
#global_path = '../../up_scale_GAN_results/upscaling_GAN_3d/final/resnet_32_to_64_log_norm/'
global_path = '../../up_scale_GAN_results/upscaling_GAN_3d/medical/32_64/'

#name = 'resnet_32_to_64'
name = 'WGAN'

params_64 = dict()
params_64['save_dir'] = global_path + name + '_' + time_str + 'checkpoints/'

resume_64, params_64 = utils.test_resume(try_resume, params_64)
print(params_64['curr_counter'])

params_64['summary_dir'] = global_path + params_64['name'] + '_' + time_str +'summary/'
params_64['save_dir'] = global_path + params_64['name'] + '_' + time_str + 'checkpoints/'

print(params_64['summary_dir'])
print(params_64['save_dir'])

Resume, the training will start from the last iteration!
10000
../../up_scale_GAN_results/upscaling_GAN_3d/medical/32_64/WGAN_32_to_64summary/
../../up_scale_GAN_results/upscaling_GAN_3d/medical/32_64/WGAN_32_to_64checkpoints/


In [9]:
wgan_64 = UpscaleCosmoGAN(params_64, upscale_WGAN_pixel_CNN, is_3d=True)
gen_samples_64 = wgan_64.upscale_image(small=gen_samples_32, checkpoint=None)
#utils.save_hdf5(gen_samples_64, 'Samples/final/resnet_32_to_64_log_norm/2_64_samples_ckpt_87k_27k.h5')
utils.save_hdf5(gen_samples_64, 'Samples/medical/32_64/2_64_samples_ckpt_10k_23k.h5')

Generator 
--------------------------------------------------
     The input X is of size (?, 16, 16, 16, 1)
     The input X is upsampled to size (?, 32, 32, 32, 1)
     The input z is of size (?, 32768)
     The input y is of size (?, 32, 32, 32, 7)
     Reshape z to (?, 32, 32, 32, 1)
     Concat X and z to (?, 32, 32, 32, 2)
     conv_over_deconv= True
     Merge input y of size(?, 32, 32, 32, 7)
     Concat x and y to (?, 32, 32, 32, 9)
     0 Inception conv(1x1,3x3,5x5) layer with 64 channels
         ReLU applied
         Size of the variables: (?, 32, 32, 32, 64)
     1 Inception conv(1x1,3x3,5x5) layer with 64 channels
         ReLU applied
         Size of the variables: (?, 32, 32, 32, 64)
     2 Inception conv(1x1,3x3,5x5) layer with 64 channels
         Residual connection
         ReLU applied
         Size of the variables: (?, 32, 32, 32, 64)
     3 Inception conv(1x1,3x3,5x5) layer with 64 channels
         ReLU applied
         Size of the variables: (?, 32, 32, 32, 6

 [*] Reading checkpoints...
INFO:tensorflow:Restoring parameters from ../../up_scale_GAN_results/upscaling_GAN_3d/medical/32_64/WGAN_32_to_64checkpoints/WGAN-10000
Checkpoint successfully loaded!
Total number of patches = 2*2*2 = 8
Current patch: column=1, row=1, height=1

Current patch: column=2, row=1, height=1

Current patch: column=1, row=2, height=1

Current patch: column=2, row=2, height=1

Current patch: column=1, row=1, height=2

Current patch: column=2, row=1, height=2

Current patch: column=1, row=2, height=2

Current patch: column=2, row=2, height=2



## 64->256

## Fake 64

In [17]:
#gen_samples_64 = utils.load_hdf5('Samples/bigstat/inception_32_to_64_upsampled_upscaling_GAN_3d_32_gen_8_disc_6_new_trans/2_64_samples_ckpt_120k_163k.h5')
#gen_samples_64 = utils.load_hdf5('Samples/final/resnet_32_to_64_log_norm/2_64_samples_ckpt_87k_27k.h5')
gen_samples_64 = utils.load_hdf5('Samples/medical/32_64/2_64_samples_ckpt_4800_12k.h5')
gen_samples_64_raw = bkd(gen_samples_64)

print('gen_samples_64 shape=', gen_samples_64.shape)
gen_samples_64 = np.expand_dims(gen_samples_64, axis=4)
print('downsampled shape=', gen_samples_64.shape)

gen_samples_64 shape= (2, 64, 64, 64)
downsampled shape= (2, 64, 64, 64, 1)


## Real 256

In [11]:
real_nbody_256 = data.load.load_dataset_file(spix=256, resolution=256, Mpch=350, scaling=1, augmentation=False, shuffle=False, forward_map=ln_fwd, patch=False, is_3d=True).get_all_data()
downsampled_64 = data.load.load_dataset_file(spix=64, resolution=256, Mpch=350, scaling=4, augmentation=False, shuffle=False, forward_map=ln_fwd, patch=False, is_3d=True)
downsampled_64 = downsampled_64.get_all_data()
downsampled_64_raw = ln_bkd(downsampled_64)
print("downsampled_64 shape: ", np.shape(downsampled_64))

Loaded all 2 histograms only once in the beginning!
Loaded all 2 histograms only once in the beginning!
Tensor down_sampler_out_3d_4:0 not found, hence creating the Op.
downsampled_64 shape:  (2, 64, 64, 64)


In [12]:
print(np.min(gen_samples_64), np.max(gen_samples_64))
print(np.min(downsampled_64), np.max(downsampled_64))

0.096531734 1.1825738
0.2091262 1.5832838


In [13]:
ns = 32
#time_str = 'upscaling_GAN_3d_gen_8_disc_6_32_new_trans'
time_str = 'log_norm'

#global_path = '../../up_scale_GAN_results/upscaling_GAN_3d/inception/upsampled_upscaling_GAN_3d_32_gen_8_disc_6_new_trans/'
global_path = '../../up_scale_GAN_results/upscaling_GAN_3d/final/resnet_64_to_256_log_norm/'

#name = 'inception_upsampled_upscaling_GAN_3d_{}'.format(ns)
name = 'resnet_64_to_256'

params_256 = dict()
params_256['save_dir'] = global_path + name + '_' + time_str + 'checkpoints/'

resume, params_256 = utils.test_resume(try_resume, params_256)
print(params_256['curr_counter'])

params_256['summary_dir'] = global_path + params_256['name'] + '_' + time_str +'summary/'
params_256['save_dir'] = global_path + params_256['name'] + '_' + time_str + 'checkpoints/'

print(params_256['summary_dir'])
print(params_256['save_dir'])

Resume, the training will start from the last iteration!
116000
../../up_scale_GAN_results/upscaling_GAN_3d/final/resnet_64_to_256_log_norm/resnet_64_to_256_log_normsummary/
../../up_scale_GAN_results/upscaling_GAN_3d/final/resnet_64_to_256_log_norm/resnet_64_to_256_log_normcheckpoints/


In [14]:
wgan_256 = UpscaleCosmoGAN(params_256, upscale_WGAN_pixel_CNN, is_3d=True)
gen_samples = wgan_256.upscale_image(small=gen_samples_64, is_3d=True, checkpoint=None)
utils.save_hdf5(gen_samples, 'Samples/final/resnet_64_to_256_log_norm/2_256_samples_ckpt_116k_resnet-87k_27k.h5')

Generator 
--------------------------------------------------
     The input X is of size (?, 8, 8, 8, 1)
     The input X is upsampled to size (?, 32, 32, 32, 1)
     The input z is of size (?, 32768)
     The input y is of size (?, 32, 32, 32, 7)
     Reshape z to (?, 32, 32, 32, 1)
     Concat X and z to (?, 32, 32, 32, 2)
     conv_over_deconv= True
     Merge input y of size(?, 32, 32, 32, 7)
     Concat x and y to (?, 32, 32, 32, 9)
     0 Inception conv(1x1,3x3,5x5) layer with 32 channels
         ReLU applied
         Size of the variables: (?, 32, 32, 32, 32)
     1 Inception conv(1x1,3x3,5x5) layer with 32 channels
         ReLU applied
         Size of the variables: (?, 32, 32, 32, 32)
     2 Inception conv(1x1,3x3,5x5) layer with 32 channels
         Residual connection
         ReLU applied
         Size of the variables: (?, 32, 32, 32, 32)
     3 Inception conv(1x1,3x3,5x5) layer with 32 channels
         ReLU applied
         Size of the variables: (?, 32, 32, 32, 32)


 [*] Reading checkpoints...
INFO:tensorflow:Restoring parameters from ../../up_scale_GAN_results/upscaling_GAN_3d/final/resnet_64_to_256_log_norm/resnet_64_to_256_log_normcheckpoints/resnet_64_to_256-116000
Checkpoint successfully loaded!
Total number of patches = 8*8*8 = 512
Current patch: column=1, row=1, height=1

Current patch: column=2, row=1, height=1

Current patch: column=3, row=1, height=1

Current patch: column=4, row=1, height=1

Current patch: column=5, row=1, height=1

Current patch: column=6, row=1, height=1

Current patch: column=7, row=1, height=1

Current patch: column=8, row=1, height=1

Current patch: column=1, row=2, height=1

Current patch: column=2, row=2, height=1

Current patch: column=3, row=2, height=1

Current patch: column=4, row=2, height=1

Current patch: column=5, row=2, height=1

Current patch: column=6, row=2, height=1

Current patch: column=7, row=2, height=1

Current patch: column=8, row=2, height=1

Current patch: column=1, row=3, height=1

Current p

Current patch: column=6, row=8, height=3

Current patch: column=7, row=8, height=3

Current patch: column=8, row=8, height=3

Current patch: column=1, row=1, height=4

Current patch: column=2, row=1, height=4

Current patch: column=3, row=1, height=4

Current patch: column=4, row=1, height=4

Current patch: column=5, row=1, height=4

Current patch: column=6, row=1, height=4

Current patch: column=7, row=1, height=4

Current patch: column=8, row=1, height=4

Current patch: column=1, row=2, height=4

Current patch: column=2, row=2, height=4

Current patch: column=3, row=2, height=4

Current patch: column=4, row=2, height=4

Current patch: column=5, row=2, height=4

Current patch: column=6, row=2, height=4

Current patch: column=7, row=2, height=4

Current patch: column=8, row=2, height=4

Current patch: column=1, row=3, height=4

Current patch: column=2, row=3, height=4

Current patch: column=3, row=3, height=4

Current patch: column=4, row=3, height=4

Current patch: column=5, row=3, he

Current patch: column=2, row=1, height=7

Current patch: column=3, row=1, height=7

Current patch: column=4, row=1, height=7

Current patch: column=5, row=1, height=7

Current patch: column=6, row=1, height=7

Current patch: column=7, row=1, height=7

Current patch: column=8, row=1, height=7

Current patch: column=1, row=2, height=7

Current patch: column=2, row=2, height=7

Current patch: column=3, row=2, height=7

Current patch: column=4, row=2, height=7

Current patch: column=5, row=2, height=7

Current patch: column=6, row=2, height=7

Current patch: column=7, row=2, height=7

Current patch: column=8, row=2, height=7

Current patch: column=1, row=3, height=7

Current patch: column=2, row=3, height=7

Current patch: column=3, row=3, height=7

Current patch: column=4, row=3, height=7

Current patch: column=5, row=3, height=7

Current patch: column=6, row=3, height=7

Current patch: column=7, row=3, height=7

Current patch: column=8, row=3, height=7

Current patch: column=1, row=4, he

In [37]:
print(np.min(gen_samples), np.max(gen_samples))
print(np.min(real_nbody_256), np.max(real_nbody_256))

0.0 2.9222386
0.0 5.203132


## ----------------------------------------------------------------------------------END-------------------------------------------------------------------

In [50]:
dataset = data.load.load_dataset_file(spix=256, resolution=256, Mpch=Mpch, forward_map=data.Dataset_file.do_nothing, patch=False, is_3d=True)
real_nbody = dataset.get_all_data()
print("Nbody dataset shape: ", np.shape(real_nbody))

Nbody dataset shape:  (2, 256, 256, 256)


In [5]:
print(np.max(real_nbody[0:]), np.max(real_nbody[1:]))
print(np.min(real_nbody[0:]), np.min(real_nbody[1:]))

185874.0 185874.0
0.0 0.0


In [41]:
y = np.log(val + 1)
print(y)
x = np.e**y - 1
print(x)

88.72283905206835
3.4028234663852686e+38


In [42]:
val = np.finfo(np.float32).max
print(val)

3.4028235e+38


In [50]:
x = 185874.0
y = 1/(1 + np.e**-x)
print(y)

x = np.log(y / (1-y))
print(x)

1.0


ZeroDivisionError: float division by zero

In [58]:
box = real_nbody[1,]
print(box.shape)

(256, 256, 256)


In [59]:
box1 = data.fmap.normalize(box)
box2 = data.fmap.un_normalize(box1)
print(box2)

[[[  2.   9.  13. ...   5.   6.   4.]
  [  2.   1.  15. ...   7.   6.   2.]
  [  6.   3.  79. ...  14.   4.   9.]
  ...
  [  9.  71.   4. ...  13.   6.   8.]
  [ 11.  15.  10. ...  34.   8.   9.]
  [  3.   7.  16. ...  10.  11.   7.]]

 [[  5.   8.  23. ...   5.   8.   5.]
  [  2.   4.  15. ...   5.  10.   8.]
  [  5.   3.   6. ...  20.  10.   6.]
  ...
  [  3.  43.   8. ...   8.   4.   6.]
  [ 21.  28.  21. ...   8.   6.   6.]
  [  6.  11.  19. ...   5.   5.   8.]]

 [[ 30.  10.  21. ...  11.  12.  13.]
  [ 19.  14.  27. ...   9.  17.  10.]
  [ 12.  26.  67. ...  12.  19.  36.]
  ...
  [  2.  73.   9. ...  11.   6.   4.]
  [ 19.  37.  20. ...  10.   7.   2.]
  [ 13.  16.  22. ...   5.   9.   4.]]

 ...

 [[  3.   5.  45. ...  10.   9.  11.]
  [  6.   6.  53. ...   3.  13.   5.]
  [  8.  10.  28. ...   3.   4.   5.]
  ...
  [  4.  10.  15. ...  14.   8.   6.]
  [  5.   4.   6. ...  10.   3.   5.]
  [  6.   4.   8. ...   3.  13.  10.]]

 [[ 23.  14.  21. ...  12.   8.  11.]
  [  6.   7.

In [60]:
print(np.max(box), np.min(box))

0.7159455 -0.00033834353
